# Paso 1: localizar el Excel y listar hojas

Objetivo de este paso:
1) Comprobar que Python ve el archivo Excel correcto.
2) Listar los nombres de las hojas **exactamente** como los detecta `pandas`

In [1]:
from pathlib import Path
import pandas as pd

# Opciones de visualización (solo para que se vea cómodo)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# 1) Ruta del Excel (debe estar en la misma carpeta del notebook)
xlsx_path = Path("Datos Brasil, España, Mexico.xlsx")

print("¿Existe el archivo?:", xlsx_path.exists())
print("Ruta absoluta:", xlsx_path.resolve())

# 2) Listar hojas
xls = pd.ExcelFile(xlsx_path)
sheet_names = xls.sheet_names
print("Hojas detectadas:", sheet_names)


¿Existe el archivo?: True
Ruta absoluta: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\Datos Brasil, España, Mexico.xlsx
Hojas detectadas: ['Mexico', 'Brazil', 'Spain']


# Paso 2: Cargar las hojas del Excel a DataFrames

En este paso vamos a leer las tres hojas detectadas:
- `Mexico` → contiene 70 pacientes.
- `Brazil` → contiene 220 pacientes (143 de Brasil y 77 de España).
- `Spain` → está vacía.

**Objetivo de este paso:**  
1. Confirmar cuántas filas y columnas tiene cada hoja.  
2. Verificar que la hoja `Spain` realmente está vacía.  

No hacemos limpieza ni cambios todavía, solo lectura y revisión del tamaño.


In [2]:
# Cargar hojas
df_mexico_raw = pd.read_excel(xlsx_path, sheet_name="Mexico")
df_brazil_raw = pd.read_excel(xlsx_path, sheet_name="Brazil")
df_spain_raw  = pd.read_excel(xlsx_path, sheet_name="Spain")

print("Mexico:", df_mexico_raw.shape)
print("Brazil:", df_brazil_raw.shape)
print("Spain:", df_spain_raw.shape)


Mexico: (70, 73)
Brazil: (220, 73)
Spain: (0, 0)


# Paso 3: Limpieza mínima de **nombres de columnas**

**Por qué hacerlo ahora:**
- Evita errores tipo `KeyError` por **espacios dobles** o **espacios al final** (ej. `"Platelets "`).
- Facilita que las equivalencias México↔Brasil funcionen sin sorpresas.
- No tocamos contenidos ni unidades (mg/dL, ng/mL…) todavía; **solo** limpiamos espacios.

**Qué haremos:**
1) Colapsar espacios múltiples a un único espacio.
2) Quitar espacios al principio y al final.
3) Aplicarlo **por igual** a `df_mexico_raw` y `df_brazil_raw`.

Al final veremos **cuántos nombres han cambiado** y un pequeño listado de ejemplos para confirmar que todo está bien.

In [3]:
import re
import pandas as pd

def clean_col_spaces(df: pd.DataFrame) -> tuple[pd.DataFrame, list]:
    """Devuelve (df_limpio, lista_de_cambios) donde lista_de_cambios son pares (antes, después)."""
    old_cols = list(map(str, df.columns))
    new_cols = [re.sub(r"\s+", " ", c).strip() for c in old_cols]
    changes = [(o, n) for o, n in zip(old_cols, new_cols) if o != n]
    return df.rename(columns=dict(zip(old_cols, new_cols))), changes

# Aplicar a ambas hojas (Spain está vacía, la ignoramos)
df_mexico = df_mexico_raw.copy()
df_brazil = df_brazil_raw.copy()

df_mexico, changes_mx = clean_col_spaces(df_mexico)
df_brazil, changes_br = clean_col_spaces(df_brazil)

print(f"Cambios en nombres (México): {len(changes_mx)}")
print(changes_mx[:10])  # mostramos solo los 10 primeros si hay muchos

print(f"\nCambios en nombres (Brasil): {len(changes_br)}")
print(changes_br[:10])

# Comprobación rápida: que 'Platelets' NO termine en espacio, y que 'Weight (kg)' no tenga dobles espacios
suspects = [c for c in df_mexico.columns if "platelet" in c.lower() or "weight" in c.lower()] + \
           [c for c in df_brazil.columns if "platelet" in c.lower() or "weight" in c.lower()]
print("\nComprobación rápida (nombres que contienen 'platelet' o 'weight'):")
print(sorted(set(suspects)))


Cambios en nombres (México): 6
[(' HCQ use (mg/day)', 'HCQ use (mg/day)'), ('Weight  (kg)', 'Weight (kg)'), ('VLDL (mg/dL) ', 'VLDL (mg/dL)'), ('Platelets ', 'Platelets'), ('VCM ', 'VCM'), ('CHCM ', 'CHCM')]

Cambios en nombres (Brasil): 8
[(' HCQ use (mg/day)', 'HCQ use (mg/day)'), ('SLEDAI ', 'SLEDAI'), ('Weight  (kg)', 'Weight (kg)'), ('VLDL (mg/dL) ', 'VLDL (mg/dL)'), ('Triglycerides (mg/dL) ', 'Triglycerides (mg/dL)'), ('Reactive C protein  (mg/dl)', 'Reactive C protein (mg/dl)'), ('VCM ', 'VCM'), ('CHCM ', 'CHCM')]

Comprobación rápida (nombres que contienen 'platelet' o 'weight'):
['Platelets', 'Weight (kg)']


# Paso 4: Estandarizar nombres equivalentes (MX ↔ BR)

**Objetivo:** Unificar *nombres* que representan la misma variable pero difieren por mayúsculas, espacios o pequeñas variantes.
> Aún **no** tocamos tipos de datos ni unidades; solo nombres.

**Casos incluidos:**
- Age (Years) ↔ Age (years)
- smoking habits ↔ Smoking habits
- Total Cholesterol ↔ Total cholesterol
- Triglycerides (con/sin espacio final)
- Uric/Creatinine/Urea (mg/dL vs mg/dl) → normalizamos a **mg/dL** en el nombre
- Folic/Vitamin D (ng/mL vs ng/ml) → normalizamos a **ng/mL** en el nombre
- Bone mass (kg/Kg)
- VLDL (espacio)
- SLEDAI (Mex-SLEDAI) ↔ SLEDAI
- SLICC_ACR ↔ SLICC
- CRP: "C reactive protein (mg/L)" ↔ "Reactive C protein (mg/dl)" → nombre canónico **C-reactive protein** (la **unidad** la gestionaremos después)

**Comprobación al final:**  
Listar qué columnas quedan **solo en México** y **solo en Brasil** tras renombrar.  
Idealmente, solo deberían quedar los **identificadores** distintos (Folio/Grupo vs Patient/RGHC).


In [4]:
# 1) Diccionario de equivalencias (solo nombres)
column_equivalences = {
    # Estándar de mayúsculas/espacios/typos
    "Age (Years)": "Age (years)",
    "smoking habits": "Smoking habits",
    "Total Cholesterol (mg/dL)": "Total cholesterol (mg/dL)",
    "Triglycerides (mg/dL) ": "Triglycerides (mg/dL)",
    "Tryglicerides (mg/dL)": "Triglycerides (mg/dL)",

    # Unidades en el nombre (normalizamos a mg/dL, ng/mL)
    "Uric Acid (mg/dL)": "Uric acid (mg/dL)",
    "Uric acid (mg/dl)": "Uric acid (mg/dL)",
    "Creatinine (mg/dL)": "Creatinine (mg/dL)",
    "Creatinine (mg/dl)": "Creatinine (mg/dL)",
    "Urea (mg/dL)": "Urea (mg/dL)",
    "Urea (mg/dl)": "Urea (mg/dL)",
    "Folic Acid (ng/mL)": "Folic acid (ng/mL)",
    "Folic acid (ng/ml)": "Folic acid (ng/mL)",
    "Vitamin D (ng/mL)": "Vitamin D (ng/mL)",
    "Vitamin D (ng/ml)": "Vitamin D (ng/mL)",
    "Bone mass (Kg)": "Bone mass (kg)",

    # CRP y VLDL
    "Reactive C protein (mg/dl)": "C-reactive protein",
    "C reactive protein (mg/L)": "C-reactive protein",
    "VLDL (mg/dL)": "VLDL (mg/dL)",  # por si acaso tras limpieza
    # Diferenciales y scores
    "SLEDAI (Mex-SLEDAI)": "SLEDAI",
    "SLEDAI": "SLEDAI",
    "SLICC_ACR": "SLICC",
    "SLICC": "SLICC",
    # Leucocitos diferenciales (unidad la veremos luego)
    "Neutrophils (%)": "Neutrophils",
    "Lymphocytes (%)": "Lymphocytes",
    "Monocytes (%)": "Monocytes",
}

# 2) Aplicar a copias estandarizadas
df_mx_std = df_mexico.rename(columns=column_equivalences).copy()
df_br_std = df_brazil.rename(columns=column_equivalences).copy()

# 3) Chequear qué columnas siguen “solo en MX” o “solo en BR”
set_mx = set(df_mx_std.columns)
set_br = set(df_br_std.columns)
solo_mx = sorted(set_mx - set_br)
solo_br = sorted(set_br - set_mx)

print("Columnas solo en México:", len(solo_mx), solo_mx[:20])
print("Columnas solo en Brasil:", len(solo_br), solo_br[:20])


Columnas solo en México: 4 ['Albumin (g/dL)', 'Folio', 'Grupo', 'Time of the disease (years)']
Columnas solo en Brasil: 4 ['Albumin /d/dl)', 'Patient', 'RGHC', 'Time of disease (years)']


# Paso 5: Completar equivalencias que faltan

**Qué unificamos ahora:**
- `Albumin (g/dL)` ↔ `Albumin /d/dl)` → usar **Albumin (g/dL)**
- `Time of the disease (years)` ↔ `Time of disease (years)` → usar **Time of disease (years)**

**Objetivo de la comprobación:**
Tras renombrar, que "solo en México" y "solo en Brasil" queden únicamente los
**identificadores** (`Folio`, `Grupo` vs `Patient`, `RGHC`).


In [5]:
# Añadimos equivalencias que faltaban
column_equivalences.update({
    "Albumin /d/dl)": "Albumin (g/dL)",
    "Time of the disease (years)": "Time of disease (years)"
})

# Reaplicar sobre copias
df_mx_std = df_mexico.rename(columns=column_equivalences).copy()
df_br_std = df_brazil.rename(columns=column_equivalences).copy()

# Recalcular diferencias de columnas
set_mx = set(df_mx_std.columns)
set_br = set(df_br_std.columns)
solo_mx = sorted(set_mx - set_br)
solo_br = sorted(set_br - set_mx)

print("Columnas solo en México:", len(solo_mx), solo_mx)
print("Columnas solo en Brasil:", len(solo_br), solo_br)


Columnas solo en México: 2 ['Folio', 'Grupo']
Columnas solo en Brasil: 2 ['Patient', 'RGHC']


# Paso 6: Crear la columna `Country` a partir de `Center`

**Por qué:** Necesitamos un indicador del país de cada paciente para análisis comparativos,
y en tus hojas `Center` ya distingue bien:  
- En *Mexico*: todos son `Mexico`.  
- En *Brazil*: aparecen `Brazil` y `Spain`.

**Qué haremos:**  
1) En cada DataFrame estandarizado (`df_mx_std`, `df_br_std`), crear `Country = Center`.  
2) Verificar frecuencias de `Country`.


In [6]:
# Asignar Country directamente desde Center
df_mx_std = df_mx_std.copy()
df_br_std = df_br_std.copy()

df_mx_std["Country"] = df_mx_std["Center"]
df_br_std["Country"] = df_br_std["Center"]

print("Frecuencias de Country — México:")
print(df_mx_std["Country"].value_counts(dropna=False))

print("\nFrecuencias de Country — Brazil:")
print(df_br_std["Country"].value_counts(dropna=False))


Frecuencias de Country — México:
Country
Mexico    70
Name: count, dtype: int64

Frecuencias de Country — Brazil:
Country
Brazil    143
Spain      77
Name: count, dtype: int64


# Paso 7: Revisar los tipos de datos antes de concatenar

**Objetivo:**  
Confirmar que las columnas comunes entre México y Brasil tienen **el mismo tipo de dato**.
- Si una columna es `float` en un DataFrame y `object` en otro, lo detectamos.
- Esto nos permitirá forzar la conversión antes de concatenar, evitando problemas de formatos raros.

**Qué haremos:**  
1. Listar el número de columnas por tipo (`int64`, `float64`, `object`) en cada DataFrame.  
2. Comparar tipos columna a columna entre México y Brasil.  
3. Identificar las discrepancias para corregirlas en el siguiente paso.


In [7]:
# Resumen de tipos en cada DataFrame
print("México - resumen de tipos:")
print(df_mx_std.dtypes.value_counts(), "\n")

print("Brasil - resumen de tipos:")
print(df_br_std.dtypes.value_counts(), "\n")

# Comparar tipos columna a columna
common_cols = sorted(set(df_mx_std.columns) & set(df_br_std.columns))
diff_types = [(c, df_mx_std[c].dtype, df_br_std[c].dtype)
              for c in common_cols if df_mx_std[c].dtype != df_br_std[c].dtype]

print("Columnas con tipos diferentes entre México y Brasil:")
for col, t_mx, t_br in diff_types:
    print(f"- {col}: México={t_mx}, Brasil={t_br}")


México - resumen de tipos:
float64    40
object     32
int64       2
Name: count, dtype: int64 

Brasil - resumen de tipos:
float64    56
object     17
int64       1
Name: count, dtype: int64 

Columnas con tipos diferentes entre México y Brasil:
- Albumin (g/dL): México=object, Brasil=float64
- BMI (kg/m2): México=object, Brasil=float64
- Bone mass (kg): México=object, Brasil=float64
- Creatinine (mg/dL): México=object, Brasil=float64
- Fat mass (%): México=object, Brasil=float64
- Folic acid (ng/mL): México=object, Brasil=float64
- HDL (mg/dL): México=object, Brasil=float64
- Height (m): México=object, Brasil=float64
- Hematocrit: México=object, Brasil=float64
- Leukocytes: México=object, Brasil=float64
- Lipid (%TEI): México=object, Brasil=float64
- Lymphocytes: México=object, Brasil=float64
- Metrotexato use (mg/day): México=float64, Brasil=object
- Monocytes: México=object, Brasil=float64
- RDW: México=object, Brasil=float64
- Total body water (%): México=object, Brasil=float64
- 

# Paso 8: Unificar tipos en columnas con discrepancias

**Objetivo:** convertir a **numérico** (float) las columnas que aparecen como `object` en un DataFrame y `float` en el otro.

**Cómo lo haremos:**
- Limpiar texto típico de Excel: comas como decimales, símbolos `%`, espacios.
- Mantener solo dígitos, signo y punto.
- Convertir con `pd.to_numeric(errors="coerce")`.

**Columnas a corregir (según tu salida):**
Albumin (g/dL), BMI (kg/m2), Bone mass (kg), Creatinine (mg/dL), Fat mass (%),
Folic acid (ng/mL), HDL (mg/dL), Height (m), Hematocrit, Leukocytes,
Lipid (%TEI), Lymphocytes, Metrotexato use (mg/day), Monocytes, RDW,
Total body water (%), Urea (mg/dL), Uric acid (mg/dL), VCM, Vitamin B12 (ng/ml)

> Nota: algunas aparecen como numéricas en Brasil y `object` en México, y un par al revés.


In [8]:
import pandas as pd
import numpy as np
import re

# 1) Función de limpieza -> numérico
def clean_to_numeric(series: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(series):
        return series
    s = series.astype(str)
    s = s.str.replace(",", ".", regex=False)   # coma -> punto
    s = s.str.replace("%", "", regex=False)    # quitar %
    # mantener dígitos, punto, signo y notación científica
    s = s.str.replace(r"[^0-9eE\+\-\.]", "", regex=True)
    s = s.replace({"": np.nan, ".": np.nan, "-": np.nan, "+": np.nan})
    return pd.to_numeric(s, errors="coerce")

# 2) Columnas a corregir (las de tu listado)
cols_to_fix = [
    "Albumin (g/dL)", "BMI (kg/m2)", "Bone mass (kg)", "Creatinine (mg/dL)",
    "Fat mass (%)", "Folic acid (ng/mL)", "HDL (mg/dL)", "Height (m)",
    "Hematocrit", "Leukocytes", "Lipid (%TEI)", "Lymphocytes",
    "Metrotexato use (mg/day)", "Monocytes", "RDW", "Total body water (%)",
    "Urea (mg/dL)", "Uric acid (mg/dL)", "VCM", "Vitamin B12 (ng/ml)"
]

# 3) Aplicar conversión en ambos DataFrames si la columna existe
df_mx_fix = df_mx_std.copy()
df_br_fix = df_br_std.copy()

for c in cols_to_fix:
    if c in df_mx_fix.columns:
        df_mx_fix[c] = clean_to_numeric(df_mx_fix[c])
    if c in df_br_fix.columns:
        df_br_fix[c] = clean_to_numeric(df_br_fix[c])

# 4) Verificación de tipos después de la conversión
print("México - resumen tipos (post):")
print(df_mx_fix.dtypes.value_counts(), "\n")

print("Brasil - resumen tipos (post):")
print(df_br_fix.dtypes.value_counts(), "\n")

# 5) Recalcular discrepancias de tipos solo en columnas comunes
common_cols = sorted(set(df_mx_fix.columns) & set(df_br_fix.columns))
diff_types_post = [(c, df_mx_fix[c].dtype, df_br_fix[c].dtype)
                   for c in common_cols if df_mx_fix[c].dtype != df_br_fix[c].dtype]

print("Columnas con tipos diferentes (post):")
for col, t_mx, t_br in diff_types_post:
    print(f"- {col}: México={t_mx}, Brasil={t_br}")


México - resumen tipos (post):
float64    58
object     14
int64       2
Name: count, dtype: int64 

Brasil - resumen tipos (post):
float64    58
object     15
int64       1
Name: count, dtype: int64 

Columnas con tipos diferentes (post):


# Paso 9: Concatenar México + Brasil y verificar

**Objetivo:** crear el dataset maestro `df_master` ya con tipos alineados.

**Verificaciones tras concatenar:**
- `shape` esperado ≈ (290, Nº columnas comunes + identificadores + Country)
- `Country` con 3 niveles: Mexico / Brazil / Spain
- Sin columnas duplicadas en nombre
- Recuento de tipos en el combinado (object vs float vs int)


In [9]:
import pandas as pd

# 1) Concatenar
df_master = pd.concat([df_mx_fix, df_br_fix], ignore_index=True)

# 2) Checks
print("Shape combinado:", df_master.shape)

dups = df_master.columns[df_master.columns.duplicated()].tolist()
print("Columnas duplicadas:", dups)

print("Valores únicos en 'Country':", df_master["Country"].unique())
print("Frecuencias 'Country':")
print(df_master["Country"].value_counts(dropna=False))

print("\nTipos en combinado:")
print(df_master.dtypes.value_counts())


Shape combinado: (290, 76)
Columnas duplicadas: []
Valores únicos en 'Country': ['Mexico' 'Brazil' 'Spain']
Frecuencias 'Country':
Country
Brazil    143
Spain      77
Mexico     70
Name: count, dtype: int64

Tipos en combinado:
float64    59
object     16
int64       1
Name: count, dtype: int64


In [10]:
df_master.head()


,Folio,Grupo,Center,Race,Gender,Age (years),Marital status,Education level,Smoking habits,Time of disease (years),HCQ use (mg/day),HCQ use (mg/kg/day),Corticoide use (mg/day),Metrotexato use (mg/day),SLICC,SLEDAI,Weight (kg),Height (m),BMI (kg/m2),Waist Circ (cm),Fat mass (kg),Fat mass (%),Fat free mass (kg),Fat free mass (%),Bone mass (kg),Total body water (%),Systolic Blood Pressure (mm/Hg),Diastolic Blood Pressure (mm/Hg),Glucose (mg/dL),Total cholesterol (mg/dL),LDL (mg/dL),HDL (mg/dL),n-HDL (mg/dL),VLDL (mg/dL),Triglycerides (mg/dL),C-reactive protein,Albumin (g/dL),Uric acid (mg/dL),Insuline (U/ml),GOT_AST (U/L),GPT_ALT (U/L),Urea (mg/dL),Creatinine (mg/dL),Folic acid (ng/mL),Vitamin B12 (ng/ml),Vitamin D (ng/mL),Leukocytes,Neutrophils,Lymphocytes,Monocytes,Platelets,Hemoglobin,Hematocrit,VCM,CHCM,RDW,VSG (mm),C3 complement,C4 complement,Anti-dsDNA,TyG,Energy intake (kcal/day),Carbohydrate intake (g/day),Carbohydrate intake (%TEI),Protein intake (g/day),Protein intake (%TEI),Lipid intake (g/day),Lipid (%TEI),METs-min/week,IPAQ,FACIT Fatigue Scale,PCS12 (HRQoL),MCS12 (HRQoL),Country,Patient,RGHC
0,1.0,Paciente,Mexico,Mexican-Mestizo,Female,29,married,Incomplete academic degree,No,4.0,NaN,NaN,NaN,NaN,0.0,4.0,95.9,1.61,36.9,111.1,NaN,46.9,NaN,NaN,NaN,39.0,143.0,66.0,75.00,175.00,73.80,40.00,NaN,NaN,306.00,8.51,NaN,7.96,NaN,NaN,NaN,35.3,NaN,8444.0,NaN,3265,11.63,73.70,NaN,NaN,261.0,45880.00,39.90,39.90,89.10,NaN,NaN,118.0,23,P,NaN,1158353.0,169554.0,5855003.0,5109967.00,1764563,3478867.0,2702959.0,NaN,Sedentar,NaN,NaN,NaN,Mexico,NaN,NaN
1,2.0,Paciente,Mexico,Mexican-Mestizo,Female,24,married,Incomplete academic degree,No,4.0,NaN,NaN,NaN,NaN,0.0,0.0,41.6,1.54,NaN,65.5,NaN,NaN,NaN,NaN,2.0,63.2,102.0,63.0,79.00,112.00,59.35,23.82,NaN,NaN,106.68,15.92,3.44,5.54,NaN,NaN,NaN,27.0,0.61,5775.0,NaN,14.16,4.36,79.75,14.23,3.67,344.5,45725.00,28.49,28.49,90.69,NaN,NaN,NaN,NaN,P,NaN,2260957.0,3014973.0,5333977.0,9760267.00,172675,7802666.0,3105941.0,NaN,Sedentar,NaN,NaN,NaN,Mexico,NaN,NaN
2,3.0,Paciente,Mexico,Mexican-Mestizo,Female,66,widow,Incomplete academic degree,No,16.0,NaN,NaN,NaN,NaN,4.0,0.0,67.9,1.49,NaN,93.7,NaN,39.8,NaN,NaN,NaN,41.4,137.0,77.0,124.98,198.99,126.34,51.13,NaN,NaN,119.04,3.59,3.87,NaN,NaN,NaN,NaN,48.0,0.97,6659.0,NaN,27.27,6.69,43.78,46.45,6.16,231.8,13.28,42.61,42.61,98.75,31.17,NaN,143.5,16.59,N,NaN,1043054.0,1290997.0,4950835.0,41.39,1587262,3475033.0,2998435.0,NaN,Sedentar,NaN,NaN,NaN,Mexico,NaN,NaN
3,4.0,Paciente,Mexico,Mexican-Mestizo,Female,29,single,Incomplete academic degree,yes,1.0,NaN,NaN,NaN,NaN,1.0,4.0,72.7,1.64,27.0,93.5,NaN,33.2,NaN,NaN,NaN,47.4,122.0,78.0,86.93,113.45,54.07,38.43,NaN,NaN,137.16,2025-12-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6889.0,NaN,33.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,786519.0,109732.0,5580641.0,31684.00,1611353,2495933.0,2856053.0,NaN,Activo,NaN,NaN,NaN,Mexico,NaN,NaN
4,5.0,Paciente,Mexico,Mexican-Mestizo,Female,40,single,Incomplete academic degree,No,13.0,NaN,NaN,NaN,NaN,1.0,0.0,60.1,1.62,NaN,79.8,NaN,NaN,NaN,NaN,NaN,47.6,114.0,77.0,97.00,157.00,78.57,66.67,NaN,NaN,66.00,3.19,3.72,NaN,NaN,NaN,NaN,25278.0,0.90,6718.0,NaN,25.13,NaN,38.40,52.70,NaN,210.0,45852.00,47.60,47.60,91.20,NaN,NaN,NaN,NaN,N,NaN,1712115.0,231027.0,5397464.0,7508567.00,175422,52.7,2770258.0,NaN,Sedentar,NaN,NaN,NaN,Mexico,NaN,NaN


# Paso 10: Exportar el dataset maestro

**Formato recomendado:**  
- **CSV** → seguro para análisis y evita problemas de formato (Excel no interpreta como fechas).  
- **Excel** → solo como copia de cortesía (si ves "#####" ensancha la columna; si ves "fechas", cambia formato a Número/General).

**Archivos a crear:**  
- `outputs/dataset_master_v3.csv`  
- `outputs/dataset_master_v3.xlsx`


In [11]:
from pathlib import Path

# Crear carpeta de salida
out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)

# Rutas
csv_out = out_dir / "dataset_master_v3.csv"
xlsx_out = out_dir / "dataset_master_v3.xlsx"

# Exportar CSV (canon para trabajar)
df_master.to_csv(csv_out, index=False)

# Exportar Excel (solo copia)
with pd.ExcelWriter(xlsx_out, engine="openpyxl") as writer:
    df_master.to_excel(writer, sheet_name="master", index=False)

print("Exportado CSV:", csv_out.resolve())
print("Exportado Excel:", xlsx_out.resolve())

# Verificación rápida al reabrir CSV
df_check = pd.read_csv(csv_out)
print("Shape al reabrir CSV:", df_check.shape)


Exportado CSV: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_master_v3.csv
Exportado Excel: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_master_v3.xlsx
Shape al reabrir CSV: (290, 76)


In [12]:
# 1) Ver los 10 mayores en Hemoglobin (o la columna que tú veas mal)
col = "Hemoglobin"
print("Top 10 valores en", col)
print(df_master[col].sort_values(ascending=False).head(10))

# 2) Ver 10 valores al azar para comparar
print("\nMuestra aleatoria (10):")
print(df_master[col].dropna().sample(min(10, df_master[col].notna().sum()), random_state=1))


Top 10 valores en Hemoglobin
69    46005.0
63    46004.0
64    46003.0
8     45975.0
37    45972.0
22    45916.0
15    45915.0
56    45912.0
7     45911.0
20    45882.0
Name: Hemoglobin, dtype: float64

Muestra aleatoria (10):
102       12.80
19     45787.00
189       13.00
69     46005.00
259       14.00
56     45912.00
209       12.00
194       11.20
12        11.87
100       12.10
Name: Hemoglobin, dtype: float64


# Paso B: Corregir valores mal interpretados (prueba con Hemoglobin)

**Problema:** valores como 45.911 se transformaron en 45911 al perder el separador decimal.

**Solución:** función `parse_locale_number` que:
- Reconoce separador de miles (12.345 → 12345).
- Reconoce coma como decimal (12,3 → 12.3).
- Convierte a float manteniendo decimales.

**Prueba:** aplicamos a `Hemoglobin` y comparamos original vs corregido.


In [13]:
import re
import numpy as np

def parse_locale_number(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()

    # Caso simple: ya es float válido
    try:
        return float(s)
    except ValueError:
        pass

    # Caso con punto y coma -> formato europeo
    if "." in s and "," in s:
        s = s.replace(".", "").replace(",", ".")
        return pd.to_numeric(s, errors="coerce")

    # Solo comas -> decimal
    if "," in s:
        s = s.replace(",", ".")
        return pd.to_numeric(s, errors="coerce")

    # Solo puntos
    if "." in s:
        # Patrón de miles (grupos de 3 dígitos)
        if re.fullmatch(r"\d{1,3}(?:\.\d{3})+", s):
            s = s.replace(".", "")
            return pd.to_numeric(s, errors="coerce")
        else:
            return pd.to_numeric(s, errors="coerce")

    # Último recurso: limpiar caracteres
    s = re.sub(r"[^0-9\.\-]", "", s)
    return pd.to_numeric(s, errors="coerce")

# Aplicar SOLO a Hemoglobin (prueba)
df_master["Hemoglobin_fix"] = df_master["Hemoglobin"].apply(parse_locale_number)

print("Comparación original vs corregido (valores grandes):")
print(df_master.loc[df_master["Hemoglobin"] > 1000, ["Hemoglobin", "Hemoglobin_fix"]].head(10))

print("\nResumen de Hemoglobin_fix:")
print(df_master["Hemoglobin_fix"].describe())


Comparación original vs corregido (valores grandes):
    Hemoglobin  Hemoglobin_fix
0      45880.0         45880.0
1      45725.0         45725.0
4      45852.0         45852.0
5      45761.0         45761.0
6      45725.0         45725.0
7      45911.0         45911.0
8      45975.0         45975.0
9      45789.0         45789.0
11     45819.0         45819.0
13     45702.0         45702.0

Resumen de Hemoglobin_fix:
count      276.000000
mean      5324.684058
std      14694.063241
min          0.090000
25%         12.400000
50%         13.400000
75%         14.400000
max      46005.000000
Name: Hemoglobin_fix, dtype: float64


In [14]:
# Ver cómo está Hemoglobin en el DataFrame ANTES de la conversión
print("Tipos antes de convertir (df_mx_std):", df_mx_std["Hemoglobin"].dtype)
print("\nEjemplo de valores originales en df_mx_std['Hemoglobin'] (10 al azar):")
print(df_mx_std["Hemoglobin"].dropna().sample(10, random_state=1).tolist())


Tipos antes de convertir (df_mx_std): float64

Ejemplo de valores originales en df_mx_std['Hemoglobin'] (10 al azar):
[45853.0, 12.23, 45912.0, 45668.0, 10.98, 13.28, 45916.0, 16.33, 14.38, 45786.0]


In [15]:
# 1) Releer desde el Excel original SOLO la columna Hemoglobin de la hoja "Mexico", como TEXTO
mx_hemo_raw = pd.read_excel("Datos Brasil, España, Mexico.xlsx",
                            sheet_name="Mexico",
                            usecols=["Hemoglobin"],
                            dtype=str)

print("Muestra cruda (texto) desde Excel → Mexico['Hemoglobin']:")
print(mx_hemo_raw.head(10).to_dict(orient="list"))

# 2) Mostrar lado a lado (primero 10 filas) lo que tienes en df_mx_fix vs lo recién leído como texto
comp = pd.DataFrame({
    "df_mx_fix (actual)": df_mx_fix["Hemoglobin"].head(10).tolist(),
    "excel_texto (releído)": mx_hemo_raw["Hemoglobin"].head(10).tolist(),
})
comp


Muestra cruda (texto) desde Excel → Mexico['Hemoglobin']:
{'Hemoglobin': ['45880', '45725', '13.28', nan, '45852', '45761', '45725', '45911', '45975', '45789']}


,df_mx_fix (actual),excel_texto (releído)
0,45880.00,45880
1,45725.00,45725
2,13.28,13.28
3,NaN,NaN
4,45852.00,45852
5,45761.00,45761
6,45725.00,45725
7,45911.00,45911
8,45975.00,45975
9,45789.00,45789


In [16]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Copiamos solo para inspección
col = "Hemoglobin"

# Rango plausible (ajústalo si tu clínica usa otros umbrales)
lower, upper = 5.0, 20.0

s = df_master[col]
# Valores no nulos que caen fuera del rango razonable
mask_improbable = s.notna() & ((s < lower) | (s > upper))

n_total = s.notna().sum()
n_bad = mask_improbable.sum()
n_ok = n_total - n_bad

print(f"Columna: {col}")
print(f"  No nulos totales: {n_total}")
print(f"  Valores en rango [{lower}, {upper}]: {n_ok}")
print(f"  Valores improbables (fuera de rango): {n_bad}")

# Ver algunos ejemplos de valores improbables
ejemplos = df_master.loc[mask_improbable, [col, "Country"]].head(10)
print("\nEjemplos de valores improbables (primeros 10):")
print(ejemplos)

# Mini-informe de calidad: intentamos inferir si el número "parece" un serial de fecha de Excel
# Excel (Windows) tiene origen 1899-12-30; 1 -> 1899-12-31. Seriales ~ 45000 son 2023-2025.
excel_epoch = datetime(1899, 12, 30)

def maybe_excel_date(n):
    try:
        if 20000 <= float(n) <= 60000:
            return (excel_epoch + timedelta(days=float(n))).date().isoformat()
        else:
            return np.nan
    except:
        return np.nan

qc_issues = df_master.loc[mask_improbable, [col, "Country"]].copy()
qc_issues["excel_like_date"] = qc_issues[col].apply(maybe_excel_date)

print("\nMuestra del informe QC (posibles seriales de fecha):")
print(qc_issues.head(10))
print("\nConteo de filas con 'excel_like_date' no nulo:", qc_issues["excel_like_date"].notna().sum())


Columna: Hemoglobin
  No nulos totales: 276
  Valores en rango [5.0, 20.0]: 243
  Valores improbables (fuera de rango): 33

Ejemplos de valores improbables (primeros 10):
    Hemoglobin Country
0     45880.00  Mexico
1     45725.00  Mexico
4     45852.00  Mexico
5     45761.00  Mexico
6     45725.00  Mexico
7     45911.00  Mexico
8     45975.00  Mexico
9     45789.00  Mexico
10        0.09  Mexico
11    45819.00  Mexico

Muestra del informe QC (posibles seriales de fecha):
    Hemoglobin Country excel_like_date
0     45880.00  Mexico      2025-08-11
1     45725.00  Mexico      2025-03-09
4     45852.00  Mexico      2025-07-14
5     45761.00  Mexico      2025-04-14
6     45725.00  Mexico      2025-03-09
7     45911.00  Mexico      2025-09-11
8     45975.00  Mexico      2025-11-14
9     45789.00  Mexico      2025-05-12
10        0.09  Mexico             NaN
11    45819.00  Mexico      2025-06-11

Conteo de filas con 'excel_like_date' no nulo: 32


# Paso C: Saneamiento de Hemoglobin

**Qué haremos:**
- Detectar valores fuera del rango [5–20] g/dL.
- Sustituirlos por `NaN` en `Hemoglobin_clean`.
- Generar un informe `hemoglobin_qc.csv` con esos casos, incluyendo la fecha probable que Excel había metido.


In [17]:
from pathlib import Path

col = "Hemoglobin"
lower, upper = 5.0, 20.0

# Creamos columna corregida
df_master[col + "_clean"] = df_master[col].where(
    (df_master[col].between(lower, upper)) | (df_master[col].isna()), 
    np.nan
)

# Informe QC para trazabilidad
qc_issues = df_master.loc[df_master[col + "_clean"].isna() & df_master[col].notna(),
                          [col, "Country"]].copy()
qc_issues["excel_like_date"] = qc_issues[col].apply(maybe_excel_date)

# Guardamos informe
out_dir = Path("outputs"); out_dir.mkdir(exist_ok=True)
qc_path = out_dir / "hemoglobin_qc.csv"
qc_issues.to_csv(qc_path, index=False)

print("Guardado informe QC en:", qc_path.resolve())
print("Resumen de Hemoglobin_clean:")
print(df_master[col + "_clean"].describe())


Guardado informe QC en: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\hemoglobin_qc.csv
Resumen de Hemoglobin_clean:
count    243.000000
mean      13.048189
std        1.497947
min        8.310000
25%       12.255000
50%       13.200000
75%       14.000000
max       17.100000
Name: Hemoglobin_clean, dtype: float64


# Paso D: Exportar dataset maestro con correcciones

**Qué incluimos:**
- Todas las columnas originales.
- + Columnas *_clean para variables corregidas (por ahora solo Hemoglobin).

**Archivos:**
- `outputs/dataset_master_clean.csv` → formato canon (seguro).
- `outputs/dataset_master_clean.xlsx` → copia para Excel.


In [19]:
# Exportar dataset con columnas originales + Hemoglobin_clean
csv_out = out_dir / "dataset_master_clean.csv"
xlsx_out = out_dir / "dataset_master_clean.xlsx"

df_master.to_csv(csv_out, index=False)

with pd.ExcelWriter(xlsx_out, engine="openpyxl") as writer:
    df_master.to_excel(writer, sheet_name="master", index=False)

print("Exportado CSV:", csv_out.resolve())
print("Exportado Excel:", xlsx_out.resolve())


Exportado CSV: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_master_clean.csv
Exportado Excel: C:\Users\manue\TFM MÁSTER BIOINFORMÁTICA\outputs\dataset_master_clean.xlsx


In [ ]:
Yo